In [10]:
%pip -q install google-genai

In [11]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [12]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [13]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [14]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [15]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [16]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [92]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model = "gemini-2.0-flash",
      description= "Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar as últimas notícias relevantes sobre a ocorrência do evento extremo inserido no tópico.
      Você deve pesquisar notícias sobre eventos extremos que já ocorreram. Não inclua notícias sobre eventos futuros.
      Você deve consultar sites de agências de notícias do mundo todo (como Folha de São Paulo, O Globo, El país, The New York Times, BBC, etc...) ou sites específicos de climatologia (como INMET, Méteo France, Climate.gov, etc)
      O tópico inserido estará escrito em português ou inglês. Você deve realizar a busca no Google (google_search) em português, inglês, espanhol, francês, italiano e chinês.
      Essas notícias devem ter sido publicadas no máximo 12 meses antes da data de hoje.
      Tente encontrar notícias de eventos extremos em diferentes países.
      Apresente a lista de notícias encontradas, garantindo que TODOS os itens da lista incluam a URL para que o próximo agente possa processá-lo.
      Idealmente encontre no mínimo 10 notícias.
      As notícias devem ter sido publicadas no máximo 12 meses antes da data de hoje. Não inclua nos resultados notícias publicadas há mais de 12 meses da data de hoje.
      Idealmente encontre notícias de diferentes países, em todos os continentes.
      Sua saída deve ser uma lista numerada. Cada item da lista é uma notícia. A lista deve ter a seguinte estrutura:
            Número da notícia: Veículo que publicou a notícia
            Data da notícia
            Título da notícia
            Resumo do que aconteceu
            URL da notícia


      """
      )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  noticias=call_agent(buscador, entrada_do_agente_buscador)
  return noticias


In [49]:
######################################
# --- Agente 2: Analisador de resposta --- #
######################################

def agente_analisador (noticias_buscadas):
    analisador = Agent(
        name="agente_analisador",
        model="gemini-2.0-flash",
        description="Agente analisador de impacto",
        tools=[],
        instruction="""
            Você é um analista dos impactos de eventos extremos.
            Sua função é analisar o texto das notícias para extrair dados quantificáveis sobre as consequências. Exemplos: número de mortos ou feridos, número de pessoas desalojadas,
            estimativa de perdas financeiras (se mencionada), área afetada (em km² ou hectares), número de casas danificadas, duração da interrupção de serviços (energia, água), etc.
            Você receberá uma lista de notícias. Para cada notícia nessa lista, siga os passos:
            1. Leia e analise o conteúdo da notícia.
            2. Identifique dados quantificáveis sobre as consequências do evento extremo analisado (topico), como por exemplo: número de mortos ou feridos, número de pessoas desalojadas,
            estimativa de perdas financeiras (se mencionada), área afetada (em km² ou hectares), número de casas danificadas, serviços interrompidos (energia, água), etc.
            3. Se encontrar os dados do item 2, liste-os de forma clara.
            4. Se não encontrar os dados do item 2, indique claramente "sem info. de impacto".

            Sua saída deve ser uma lista, contendo todos os itens da lista de noticias_buscadas. Cada item da lista deve ter a seguinte estrutura:
            Título da notícia
            URL da notícia
            Analise de impacto


            Certifique-se de que a sua resposta seja APENAS o título da notícia, a URL e a análise e impacto, sem texto adicional antes ou depois.

               """,

    )
    entrada_do_agente_analisador = f"Tópico: {topico}\nNoticias_buscadas: {noticias_buscadas}"
    # Executa o agente
    analise = call_agent(analisador, entrada_do_agente_analisador)
    return analise



In [107]:
###############################################
# --- Agente 3: Classificador de Notícias --- #
###############################################

# Função agente_classificador

def agente_classificador (noticias_buscadas, noticas_analisadas):
    classificador = Agent(
        name="agente_classificador",
        model="gemini-2.0-flash",
        description="Agente que tabula notícias sobre eventos extremos.",
        tools=[],
        instruction="""
        Você é um organizador de informações.
        Sua tarefa é tabular as informações sobre os eventos extremos encontrados em noticias_buscadas e em noticias_analisadas.
        Para cada notícia encontrada, extraia as seguintes informações:
        1. Evento extremo: (Nome do evento extremo mencionado: onda de calor, inundação, seca, etc.)
        2. Localização: (Cidade, Região ou País)
        3. Data/Período: (Data ou período mencionado na notícia)
        4. Descrição: (Breve resumo do que aconteceu)
        5. Analise de impacto: (Copie a análise de impacto que consta em noticias_analisadas)
        6. Link: (Informe a URL de acesso fornecido para esta notícia específica)


        Apresente os resultados em forma de tabela, contendo todas as notícias encontradas. Cada linha da tabela é uma notícia. Cada coluna é um dos itens acima (1 a 6).
        O número de linas da tabela deve ser o mesmo número de itens da lista de noticias_buscadas.
        A tabela deve estar em português.

      """,
    )

    entrada_do_agente_classificador = f"Noticias buscadas: {noticias_buscadas}\nNoticias analisadas: {noticas_analisadas}"
    # Executa o agente
    classificacao = call_agent(classificador, entrada_do_agente_classificador)
    return classificacao


In [108]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🥵 Bem-vind@ ao Sistema de Busca e Análise de Eventos Extremos\n Aqui você encontrará notícias dos últimos 12 meses sobre eventos extremos no mundo")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o EVENTO EXTREMO sobre o qual você quer pesquisar:\n O sistema aceita termos em portugês ou inglês ")

# --- Função para verificar se o tópico é um evento extremo ---
def eh_evento_extremo(termo_busca):
    termos_permitidos = [
        "onda de calor", "calor extremo", "inundação", "enchente",
        "seca", "seca extrema", "tempestade", "furacão", "tufão", "ciclone",
        "terremoto", "erupção vulcânica", "alagamento", "vendaval",
        "tornado", "deslizamento", "chuva forte", "granizo",
        "incêndio florestal", "nevasca", "geada forte", "chuva extrema",
        "heatwave", "extreme heat", "flood", "flooding",
        "drought", "extreme drought", "storm", "hurricane", "typhoon", "cyclone",
        "earthquake", "volcanic eruption", "gale", "strong wind",
        "tornado", "landslide", "mudslide", "heavy rain", "hail",
        "wildfire", "forest fire", "blizzard", "severe frost", "hard freeze",
        "extreme rain", "extreme rainfall"
        ]

    termo_busca_lower = termo_busca.lower() # Converte para minúsculas para facilitar a comparação

    for termo_permitido in termos_permitidos:
        # Verifica se o termo permitido está contido na busca do usuário
        if termo_permitido in termo_busca_lower:
            return True # Encontrou um termo relacionado a evento extremo

    return False # Não encontrou nenhum termo relacionado a evento extremo

# --- Inserir lógica do sistema de agentes  ---
########################################################################

if not topico:
    print("Você esqueceu de digitar o tópico! Por favor, tente novamente.")
elif not eh_evento_extremo(topico):
    print(f"'{topico}' parece não ser um evento extremo reconhecido. O sistema só processa buscas por eventos como 'inundação', 'onda de calor', 'seca',etc.")
    print("Por favor, digite um termo relacionado a um evento extremo.")
else:
    # Se chegou aqui, o tópico não está vazio E é um evento extremo
    print(f"Maravilha! Vamos então pesquisar sobre {topico}")


    noticias_buscadas = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultados do Agente 1 (Buscador)---\n")
    print(noticias_buscadas)

    noticias_analisadas = agente_analisador(noticias_buscadas)
    print("\n--- Resultados do Agente 2 (Analisador)---\n")
    print(noticias_analisadas)

    noticias_classificadas = agente_classificador(noticias_buscadas, noticias_analisadas)
    print("\n--- Resultados do Agente 3 (Classificador)---\n")
    print(noticias_classificadas)



🥵 Bem-vind@ ao Sistema de Busca e Análise de Eventos Extremos
 Aqui você encontrará notícias dos últimos 12 meses sobre eventos extremos no mundo
❓ Por favor, digite o EVENTO EXTREMO sobre o qual você quer pesquisar:
 O sistema aceita termos em portugês ou inglês onda de calor
Maravilha! Vamos então pesquisar sobre onda de calor

--- Resultados do Agente 1 (Buscador)---

Certo, aqui estão as notícias sobre ondas de calor publicadas nos últimos 12 meses, de acordo com minhas pesquisas:


1.  **AP News:** Onda de calor no sul do Brasil leva cinco cidades a suspenderem as aulas
    *   Data: 06 de março de 2025
    *   Cinco cidades no sul do Brasil suspenderam as aulas devido a uma onda de calor, com temperaturas chegando a 36 graus Celsius. A falta de estrutura nas escolas para lidar com o calor extremo foi um dos motivos. O Instituto Nacional de Meteorologia emitiu um alerta de clima severo para o Rio Grande do Sul e outros quatro estados brasileiros.
    *   URL: [https://vertexaisear

In [109]:
###############################################################
# --- Código para processar a tabela de texto e salvar arquivo ---
###############################################################

import pandas as pd
import io
from google.colab import files # Importa a função de download do Colab

# A variável 'noticias_classificadas' já contém o texto da tabela do Agente 3

print("\n--- Tentando salvar a tabela em arquivo CSV ---")

# --- PARSING DO TEXTO DA TABELA ---
# Esta é a parte que pode precisar de ajuste dependendo de como o Agente 3
# formatou EXATAMENTE a tabela em texto.
# Usamos 'io.StringIO' para tratar a string de texto como se fosse um arquivo para o pandas ler.

data_io = io.StringIO(noticias_classificadas)

try:
    # Tentamos ler o texto usando o pandas.
    # A parte mais crucial aqui é o parâmetro 'sep' (separador).
    # Você precisará verificar a saída de texto do Agente 3:
    # - Se as colunas estiverem separadas por vírgula, use sep=','
    # - Se for ponto e vírgula, use sep=';'
    # - Se for tabulação (um bom padrão para texto tabulado), use sep='\t'
    # - Se for o pipe '|' (como tabelas Markdown), use sep='|'. Neste caso, pode ser
    #   necessário remover colunas vazias que o '|' no início/fim cria:
    #   df = pd.read_csv(data_io, sep='|').iloc[:, 1:-1].dropna(axis=1, how='all')
    # - Se for múltiplos espaços, use sep='\s+' (regex para um ou mais espaços) - cuidado com textos com espaços!

    # Vamos começar tentando um separador comum como a tabulação ('\t') ou deixando o pandas inferir
    # Ou, se a saída do seu Agente 3 parece uma tabela Markdown, tente sep='|' e ajuste
    # Vamos tentar com um separador genérico como espaço ou tabulação, que é comum em texto de tabelas.
    # Se o agente incluiu cabeçalho, 'header='infer'' (padrão) deve funcionar.
    # Se não incluiu, use header=None e defina os nomes depois: df.columns = ['Col1', 'Col2', ...]

    # **AJUSTE ESTA LINHA ('sep=') CONFORME A SAÍDA REAL DO SEU AGENTE 3**
    # Veja o texto que ele gerou e identifique o que separa as colunas.
    data_io.seek(0) # Volta para o início do "arquivo" de texto
    df = pd.read_csv(data_io, sep='\t') # <- Tente '\t', ',', ';', '|', '\s+' aqui

    # Se usar sep='|', a linha abaixo pode ajudar a limpar colunas extras:
    # df = df.iloc[:, 1:-1].dropna(axis=1, how='all')

    print("Texto da tabela lido com sucesso pelo pandas.")
    print(f"DataFrame criado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
    # Opcional: descomente a linha abaixo para ver as primeiras linhas do que o pandas leu
    # print(df.head())


    # --- SALVANDO PARA ARQUIVO CSV ---
    nome_arquivo_csv = "eventos_extremos_tabela.csv"
    # 'index=False' evita que o pandas escreva o número da linha como uma coluna extra
    df.to_csv(nome_arquivo_csv, index=False, encoding='utf-8')

    print(f"\nTabela salva como '{nome_arquivo_csv}'")

    # --- DISPONIBILIZANDO PARA DOWNLOAD NO COLAB ---
    # Esta função faz o navegador baixar o arquivo
    files.download(nome_arquivo_csv)
    print(f"Arquivo '{nome_arquivo_csv}' pronto para download.")

except Exception as e:
    # Captura qualquer erro que aconteça durante a leitura ou salvamento
    print(f"\nERRO DURANTE O PROCESSAMENTO DA TABELA OU SALVAMENTO:")
    print(f"Detalhes do erro: {e}")
    print("\nPossíveis causas:")
    print("- O formato exato da tabela de texto gerada pelo Agente 3 não pôde ser interpretado automaticamente.")
    print("- O 'separador' (sep=...) usado na linha `pd.read_csv` acima não corresponde ao caractere que o Agente 3 usou para separar as colunas.")
    print("- A tabela gerada pelo agente tem um formato inconsistente entre as linhas.")
    print("\n**Ajuste Necessário:** Por favor, examine a saída de texto *real* do Agente 3. Identifique o caractere que ele usou para separar as colunas e altere o parâmetro 'sep=' na linha `df = pd.read_csv(data_io, sep='\t')` neste código para o separador correto (por exemplo, sep=',', sep=';', sep='|', sep='\s+' para espaços).")

print("\nSistema finalizado.")


--- Tentando salvar a tabela em arquivo CSV ---
Texto da tabela lido com sucesso pelo pandas.
DataFrame criado com 14 linhas e 1 colunas.

Tabela salva como 'eventos_extremos_tabela.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo 'eventos_extremos_tabela.csv' pronto para download.

Sistema finalizado.
